# Optimizing average Jaccard distance in two dimensions

I have been considering the problem of minimizing the Jaccard distance in two dimensions. Ad-hoc visual analysis and comparison on 2d projected data is a common step in bioinformatics. As shown in [1], most dimensionality reduction methods don't preserve Jaccard distance, especially in two dimensions. Minimizing AJD in 2d reduces the risk of spurious conclusions resulting from neighborhood distortion.

The approach presented in [1] finds a lower dimensional projection that minimizes the AJD, where the lower dimension is hypothesized to represent the true dimensionality of the underlying manifold.

Given that the Jaccard distance is non-smooth and discontinuous, it's quite difficult to directly optimize. In order to optimize, we first project to 2d using fast, standard techniques like PCA. We then convert the discrete neighborhood constraint into an inequality, and optimize the resulting expression with gradient descent.

[1] https://www.biorxiv.org/content/10.1101/689851v2

### Gradient descent for AJD

$k$ - number of neighbors

$a$ - source point. Point for which we determine the neighborhood in the original dimensionality

$t$ - threshold point. For $k$ neighbors, this is the $k$'th farthest point from $a$, which defines the radius of the neighborhood of $a$

$n$ - point in the neighborhood of $a$ in the original space

$\bar{n}$ - point **not** in the neighborhood of $a$ in the original space

$d(x,y)$ - Euclidean distance between points $x$ and $y$

In the lower dimensional space, $n$ satisfies the constraint if:

$$ d(a, t) > d(a, n) $$

In the lower dimensional space, the non-neighbor point $\bar{n}$ satisfies the constraint if: 

$$ d(a, t) < d(a, \bar{n}) $$

We can perform gradient descent by taking the gradient of the expression above, with respect to ${a,t,n}$ and updating their position. 

We define the gradient to be zero when the neighborhood constraint is met

$$ \nabla_a [d(a,t) - d(a, n)] = \frac{a-t}{||a-t||} - \frac{a-n}{||a-n||}$$ 
$$ \nabla_t [d(a,t) - d(a, n)] = \frac{a-t}{||a-t||}$$ 
$$ \nabla_n [d(a,t) - d(a, n)] =-\frac{a-n}{||a-n||}$$ 

At each update step, and for each point in the dataset, we sum the gradients, and apply a gradient descent step with $\epsilon$ learning rate

Below is an image demonstrating a gradient update step for a single point

![Figure illustrating neighborhood distortion and gradient update step](gradient.png)

Below is an animation of gradient descent steps with the following parameters:

`
eps = 1.0
k = 10
num_steps = 100
`

Original Average Jaccard Distance = `.890`

Final Average Jaccard Distance = `.693`

![](constraints_animation_1.gif)

Below is an animation of gradient descent steps with the following parameters:

`
eps = 0.25
k = 50
num_steps = 100
`

Original Average Jaccard Distance = `.852`

Final Average Jaccard Distance = `.663`

![](constraints_animation_2.gif)